In [1]:
import torch
import numpy as np

In [2]:
path = "/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/"
new_save_dir = "/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test_new/"

In [3]:
import os
import glob
file_list = glob.glob(os.path.join(path, "*.npz"))
file_list = [f for f in file_list if "climatology" not in f]

In [4]:
years = list(range(2017, 2019))
file_list_by_years = [[] for _ in years]
for file_name in file_list:
    year = int((file_name.split("/")[-1]).split("_")[0])
    year_index = year - years[0]
    file_list_by_years[year_index].append(file_name)

In [5]:
def sort_func(file_name):
    index = int(((file_name.split("/")[-1]).split("_")[1]).split(".")[0])
    return index
for file_list_by_year in file_list_by_years:
    file_list_by_year.sort(key = sort_func, reverse=False)

In [6]:
file_list_by_years

[['/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_0.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_1.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_2.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_3.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_4.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_5.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_6.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_7.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_8.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_9.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era5/5.625deg_npz/test/2017_10.npz',
  '/data0/datasets/weatherbench/data/weatherbench/era

In [ ]:
low_threshold = torch.load("./low_threshold.pt").numpy()
high_threshold = torch.load("./high_threshold.pt").numpy()
low_threshold = np.squeeze(low_threshold, axis=0)
high_threshold = np.squeeze(high_threshold, axis=0)

In [ ]:
time_horizon = 7 * 24
for file_list in file_list_by_years:
    yearly_data = {}
    n_instances_in_shard = 0
    for file in file_list:
        data = np.load(file)
        if yearly_data == {}:
            yearly_data = data
            random_key = next(iter(data.keys()))
            n_instances_in_shard = data[random_key].shape[0]
        else:
            yearly_data = {k: np.concatenate((yearly_data[k], data[k]), axis=0) for k in yearly_data.keys()}
            random_key = next(iter(data.keys()))
            assert n_instances_in_shard == data[random_key].shape[0]
    air_temp = yearly_data["2m_temperature"]
    mean_tensor = []
    for i in range(time_horizon, air_temp.shape[0]):
        curr_mean = np.mean(air_temp[i-time_horizon: i], axis=0)
        mean_tensor.append(curr_mean)
    mean_tensor = np.stack(mean_tensor, axis=0)

    l_mean_tensor = np.roll(mean_tensor, 1, -1)
    r_mean_tensor = np.roll(mean_tensor, -1, -1)
    d_mean_tensor = np.roll(mean_tensor, 1, -2)
    u_mean_tensor = np.roll(mean_tensor, -1, -2)

    ld_mean_tensor = np.roll(l_mean_tensor, 1, -2)
    lu_mean_tensor = np.roll(l_mean_tensor, -1, -2)
    rd_mean_tensor = np.roll(r_mean_tensor, 1, -2)
    ru_mean_tensor = np.roll(r_mean_tensor, -1, -2)

    g_mean_tensor = 4 * mean_tensor
    g_mean_tensor += l_mean_tensor + r_mean_tensor + d_mean_tensor + u_mean_tensor
    g_mean_tensor += 0.25 * (ld_mean_tensor + lu_mean_tensor + rd_mean_tensor + ru_mean_tensor)
    g_mean_tensor = g_mean_tensor / 9

    threshold_instances = np.zeros_like(air_temp[0], dtype = air_temp.dtype)
    air_temp_extreme_mask = np.zeros_like(air_temp, dtype = air_temp.dtype)
    for i in range(time_horizon, air_temp.shape[0]):
        curr_g_mean = g_mean_tensor[i - time_horizon]
        curr_mask = np.logical_or(curr_g_mean < low_threshold, curr_g_mean > high_threshold).astype(air_temp.dtype)
        air_temp_extreme_mask[i] = curr_mask
        threshold_instances += curr_mask
    n_instances = np.min(threshold_instances)
    yearly_data["2m_temperature_extreme_mask"] = air_temp_extreme_mask
    
    for shard_id, file in enumerate(file_list):
        start_index = shard_id * n_instances_in_shard
        end_index = start_index + n_instances_in_shard
        new_file_name = os.path.join(new_save_dir, file.split("/")[-1])
        sharded_data = {k: yearly_data[k][start_index:end_index] for k in yearly_data.keys()}
        np.savez(new_file_name, **sharded_data)

    print(air_temp_extreme_mask.sum(), air_temp.shape[0]*air_temp.shape[-1]*air_temp.shape[-2])



In [ ]:
1856725.0/17891328

In [7]:
newfile_list = glob.glob(os.path.join(new_save_dir, "*.npz"))
newfile_list = [f for f in newfile_list if "climatology" not in f]
years = list(range(2017, 2019))
newfile_list_by_years = [[] for _ in years]
for file_name in newfile_list:
    year = int((file_name.split("/")[-1]).split("_")[0])
    year_index = year - years[0]
    newfile_list_by_years[year_index].append(file_name)
for newfile_list_by_year in newfile_list_by_years:
    newfile_list_by_year.sort(key = sort_func, reverse=False)

In [8]:
for newfile_list, file_list in zip(newfile_list_by_years, file_list_by_years):
    for new_file, file in zip(newfile_list, file_list):
        new_data = np.load(new_file)
        data = np.load(file)
        for k in new_data.keys():
            if k == "2m_temperature_extreme_mask":
                continue
            else:
                assert (new_data[k] == data[k]).all()

In [10]:
(new_data["2m_temperature"] == data["2m_temperature"]).all()

True

In [14]:
new_data["2m_temperature_extreme_mask"][0][0].sum()

196.0

In [15]:
32*64

2048